In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [2]:
def present (image, fig):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=fig)
    plt.imshow(image)
    return image

In [3]:
def canny(image):
    image_gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    smoothed_image = cv2.GaussianBlur(image_gray, (3, 3), 0)
    bilateral1 = cv2.bilateralFilter(smoothed_image, 20, 12, 10)
    result = cv2.Canny(image=bilateral1, threshold1=200, threshold2=255)
    return result 

In [4]:
def region_of_interest(image, height, width):
    mask_temp = np.zeros((image.shape[0],image.shape[1]), np.uint8)
    temp_height= int(height)
    temp_width= int(width) 
    points = np.array([[temp_width,temp_height], [0, int(image.shape[0])], [int(image.shape[1]), int(image.shape[0])]])
    mask= cv2.fillPoly(mask_temp, pts=[points], color=(255, 255, 255))
    #plt.imshow(mask)
    detected_output = cv2.bitwise_and(image, image, mask =  mask).astype(np.uint8) 
    return detected_output

In [56]:
def draw_lines(img, lines, color=[255, 0, 0], thickness=7):
    
     #list to get positives and negatives values
    
    x_bottom_pos = []
    x_upperr_pos = []
    x_bottom_neg = []
    x_upperr_neg = []
    
    y_bottom = 540
    y_upperr = 315
    
    #y1 = slope*x1 + b
    #b = y1 - slope*x1 
    #y = slope*x + b
    #x = (y - b)/slope
    
    slope = 0
    b = 0
    
    #get x upper and bottom to lines with slope positive and negative    
    for line in lines:
        for x1,y1,x2,y2 in line:
            if (x2-x1)==0:
                continue
            #test and filter values to slope
            if ((y2-y1)/(x2-x1)) > 0.5 and ((y2-y1)/(x2-x1)) < 0.8 :
                
                slope = ((y2-y1)/(x2-x1))
                b = y1 - slope*x1
                
                x_bottom_pos.append((y_bottom - b)/slope)
                x_upperr_pos.append((y_upperr - b)/slope)
                                      
            elif ((y2-y1)/(x2-x1)) < -0.3 and ((y2-y1)/(x2-x1)) > -0.8:
            
                slope = ((y2-y1)/(x2-x1))
                b = y1 - slope*x1
                
                x_bottom_neg.append((y_bottom - b)/slope)
                x_upperr_neg.append((y_upperr - b)/slope)
                
               
    #creating a new 2d array with means
    lines_mean = np.array([[int(np.mean(x_bottom_pos)), int(np.mean(y_bottom)), int(np.mean(x_upperr_pos)), int(np.mean(y_upperr))],
                           [int(np.mean(x_bottom_neg)), int(np.mean(y_bottom)), int(np.mean(x_upperr_neg)), int(np.mean(y_upperr))]])
    
    
    #Drawing the lines
    for i in range(len(lines_mean)):
        cv2.line(img, (lines_mean[i,0], lines_mean[i,1]), (lines_mean[i,2], lines_mean[i,3]), color, thickness)

In [57]:
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap, image):

    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    lines_detected_rgb = cv2.cvtColor(np.copy(image), cv2.COLOR_BGR2RGB)
    for i in range(len(lines)):
        for x1,y1,x2,y2 in lines[i]:
            cv2.line(lines_detected_rgb,(x1,y1),(x2,y2),(0,0,255),2)
    #present(lines_detected_rgb, (10,8))
    line_img = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines )
    
    #present(image, (10,8))

    return line_img
def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return cv2.addWeighted(initial_img, α, image, β, λ)

In [58]:
cap = cv2.VideoCapture(r'C:\Users\Asus\Desktop\AUT\semister 3\vision\EX\2\CV_HW2\vid1.mp4')
rho = 1
theta = 1*np.pi/180
threshold= 8
min_line_len= 13
max_line_gap= 4




# Get current width of frame
height = int(cap.get(3))
# Get current height of frame
width = int(cap.get(4))
FILE_OUTPUT = r'C:\Users\Asus\Desktop\AUT\semister 3\vision\EX\2\CV_HW2\output80.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(FILE_OUTPUT,fourcc, 25, (height,width))


video_lengh=int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
for i in range(video_lengh):
    _ , frame = cap.read()
    edges = canny(frame)

    detected_output = region_of_interest(edges,  frame.shape[0]/2+25, frame.shape[1]/2)
    frame_lines_detected = np.copy(detected_output)
    frame_copy = np.copy(frame)
    frame_lines = hough_lines(frame_lines_detected, rho, theta,
                              threshold, min_line_len, max_line_gap, frame_copy)
    lines_edges = weighted_img(frame_lines, frame, α=0.8, β=1., λ=0.)
    out.write(lines_edges)
    

cap.release()
out.release()